### Price optimized daily diet

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as opt

from typing import List, Tuple, Dict

#### Initial data

In [2]:
# all params in 100 g
data_full = pd.read_csv(r'nutrition_eng.csv')
data_full

,name,kcal,protein,fat,carbo,price
0,buckwheat,332.9,12.6,3.3,63.2,11
1,chicken,102.5,20.0,2.5,0.0,30
2,macaroni,345.7,11.0,1.3,72.5,10
3,cucumbers,15.2,0.8,0.0,3.0,10
4,vegetable oil,899.1,0.0,99.9,0.0,15
5,rice,324.6,7.1,1.0,71.8,11
6,fish,95.6,18.5,2.4,0.0,30
7,cheese,353.0,26.0,27.0,1.5,60
8,curds,101.4,18.0,1.8,3.3,50
9,bean,60.0,5.0,0.0,10.0,20


In [3]:
data_full.name.values

array(['buckwheat', 'chicken', 'macaroni', 'cucumbers', 'vegetable oil',
       'rice', 'fish', 'cheese', 'curds', 'bean', 'bread', 'egg'],
      dtype=object)

#### Constants

In [4]:
# nutrient energetic capacity in kcal/g
PROTEIN_1g_kcal = 4.1
FAT_1g_kcal = 9.2
CARBO_1g_kcal = 4.1

all_1g_kcal = np.array([PROTEIN_1g_kcal, FAT_1g_kcal, CARBO_1g_kcal]).reshape(-1,1)

#### Constraints

In [5]:
energy_daily = 2500 # kcal
nutrient_energetic_ratios = [0.15, 0.3, 0.55] # [protein, fat, carbo]

#### Selected data

In [7]:
names_selected = np.array(['buckwheat', 'chicken', 'vegetable oil', 'bread', 'egg'])
data_selected = data_full.query('name in @names_selected')
data_selected

,name,kcal,protein,fat,carbo,price
0,buckwheat,332.9,12.6,3.3,63.2,11
1,chicken,102.5,20.0,2.5,0.0,30
4,vegetable oil,899.1,0.0,99.9,0.0,15
10,bread,233.2,7.6,0.8,48.9,15
11,egg,74.0,6.3,5.2,0.4,6


In [8]:
class PriceOptimizer:
    
    def __init__(self):
        self.diets = {}
    
    def define_constraints(self, energy: int, nutrient_ratios: List[float]) -> None:
        energy_daily = np.array([energy]).reshape(1,1)
        nutrient_energetic_ratios = np.array(nutrient_ratios).reshape(-1,1) # nutrient energetic fractions
        nutrient_mass = energy_daily * nutrient_energetic_ratios / all_1g_kcal
        self.beq = np.vstack([energy_daily, nutrient_mass]).round(1)
        
    def optimize_price(self, data: pd.DataFrame, bounds: Tuple[float, float]) -> None:
        self.table = data[['name']]
        self.price = data.price.values.reshape(-1,1) # in rub
        self.Aeq = data.iloc[:,1:5].T.values
        self.res = opt.linprog(self.price, A_eq=self.Aeq, b_eq=self.beq, method='simplex', bounds=bounds)
        
    def res_out(self, diet_key: str) -> Dict[str, pd.DataFrame]:
        self.nutrition_amount_opt = self.res.x.reshape(-1,1)
        optimal_nutrition = self.table
        val = (100 * self.nutrition_amount_opt).round(1)
        optimal_nutrition.insert(len(optimal_nutrition.columns), column='opt_amount', value=val)
        self.diets[diet_key] = optimal_nutrition
        return self.diets
    
    def print(self, vect: np.array) -> None:
        out = pd.DataFrame(vect, index=['energy, kcal', 'protein, g', 'fat, g', 'carbo, g'], columns=[''])
        print(out)
        print()

In [9]:
opter = PriceOptimizer()

In [10]:
opter.define_constraints(energy_daily, nutrient_energetic_ratios)
opter.print(opter.beq)

                    
energy, kcal  2500.0
protein, g      91.5
fat, g          81.5
carbo, g       335.4



In [11]:
opter.optimize_price(data_selected, (0,2))

In [12]:
opter.res_out('a')['a']

,name,opt_amount
0,buckwheat,200.0
1,chicken,192.5
4,vegetable oil,58.1
10,bread,200.0
11,egg,200.0


In [13]:
opter.print((opter.Aeq@opter.nutrition_amount_opt).round(1))
print(f'cost_daily, rub: {round((opter.price.T@opter.nutrition_amount_opt).item(), 2)}')

                    
energy, kcal  2000.3
protein, g      91.5
fat, g          81.5
carbo, g       225.0

cost_daily, rub: 130.47
